In [104]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3 as sql
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [105]:
connection = sql.connect('database.sqlite')

In [106]:
query = '''SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000'''

In [107]:
df = pd.read_sql_query(query,connection)

In [108]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
4995,5423,B00622CYVS,A17ASMX6QMO6XY,E. Harvill,0,1,2,1277424000,Not so tasty...,"My baby didn't seem into these dinners, so I t..."
4996,5424,B00622CYVS,A32DHN8U74GCAR,"Granola Girl ""michele j.""",0,1,4,1240790400,Food Delivery,This is great! Organic baby food options - de...
4997,5425,B00622CYVS,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0,1,5,1236988800,Dinner time is Earths Best TIme !!,My little guy loves to try new foods..so this ...
4998,5426,B00622CYVS,A2NYT3UXUTBY23,C&GHoll,1,3,2,1249603200,Wrong item shipped,We ordered the Earth's best 2nd dinner variety...


In [109]:
df['Score']

0       5
1       1
2       4
3       2
4       5
       ..
4995    2
4996    4
4997    5
4998    2
4999    1
Name: Score, Length: 5000, dtype: int64

In [110]:
filtered_score_from_query = df['Score']

In [111]:
filtered_score_from_query

0       5
1       1
2       4
3       2
4       5
       ..
4995    2
4996    4
4997    5
4998    2
4999    1
Name: Score, Length: 5000, dtype: int64

In [112]:
def score_rating(x):
    if x < 3:
        return 'Negative'
    return 'Positive'

In [113]:
Replaced_with_postiveNegative = filtered_score_from_query.map(score_rating)

In [114]:
Replaced_with_postiveNegative

0       Positive
1       Negative
2       Positive
3       Negative
4       Positive
          ...   
4995    Negative
4996    Positive
4997    Positive
4998    Negative
4999    Negative
Name: Score, Length: 5000, dtype: object

In [115]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
4995,5423,B00622CYVS,A17ASMX6QMO6XY,E. Harvill,0,1,2,1277424000,Not so tasty...,"My baby didn't seem into these dinners, so I t..."
4996,5424,B00622CYVS,A32DHN8U74GCAR,"Granola Girl ""michele j.""",0,1,4,1240790400,Food Delivery,This is great! Organic baby food options - de...
4997,5425,B00622CYVS,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0,1,5,1236988800,Dinner time is Earths Best TIme !!,My little guy loves to try new foods..so this ...
4998,5426,B00622CYVS,A2NYT3UXUTBY23,C&GHoll,1,3,2,1249603200,Wrong item shipped,We ordered the Earth's best 2nd dinner variety...


In [116]:
df['Score'] = Replaced_with_postiveNegative

In [117]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
4995,5423,B00622CYVS,A17ASMX6QMO6XY,E. Harvill,0,1,Negative,1277424000,Not so tasty...,"My baby didn't seem into these dinners, so I t..."
4996,5424,B00622CYVS,A32DHN8U74GCAR,"Granola Girl ""michele j.""",0,1,Positive,1240790400,Food Delivery,This is great! Organic baby food options - de...
4997,5425,B00622CYVS,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0,1,Positive,1236988800,Dinner time is Earths Best TIme !!,My little guy loves to try new foods..so this ...
4998,5426,B00622CYVS,A2NYT3UXUTBY23,C&GHoll,1,3,Negative,1249603200,Wrong item shipped,We ordered the Earth's best 2nd dinner variety...


In [118]:
df.shape

(5000, 10)

In [119]:
df.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [120]:
q = '''SELECT UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*) FROM Reviews '''

In [121]:
q = pd.read_sql_query(q,connection)

In [122]:
q

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,A3LGQPJCZVL9UC,B001LR2CU2,srfell17,1338422400,5,"I am very satisfied ,product is as advertised,...",568454


In [126]:
#query_1 = '''SELECT UserId,ProductId,ProfileName,Time,Score,Text,COUNT(UserId) FROM Reviews GROUP BY UserId HAVING COUNT(UserId)>1'''

df1 = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", connection)

In [127]:
df1

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B007Y59HVM,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ET0,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B007Y59HVM,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ET0,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBE1U,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2
...,...,...,...,...,...,...,...
80663,AZZNK89PXD006,B0039GMZZK,Etude,1269648000,5,Regular Chocolate Pocky is good on its own but...,6
80664,AZZTH6DJ0KSIP,B003Z4F56C,Crystal Caccamo,1337558400,5,i eat paleo so noodles are not allowed. i trie...,3
80665,AZZU1VEO8KUXH,B003FDG4K4,"Mia P ""Mia P""",1317513600,5,I bought this for my 13 year old daughter who ...,3
80666,AZZU4D6TZ2L6J,B0029XEXZE,"Sherry King ""llamasmama""",1247875200,5,My father thought this was the best cheese eve...,2


In [130]:
df1.loc[df1['UserId'] == 'AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B006P7E5ZI,"undertheshrine ""undertheshrine""",1334707200,5,I was recommended to try green tea extract to ...,5


In [131]:
df1['COUNT(*)']

0        2
1        3
2        2
3        3
4        2
        ..
80663    6
80664    3
80665    3
80666    2
80667    3
Name: COUNT(*), Length: 80668, dtype: int64

In [132]:
df1['COUNT(*)'].sum()

393063

# Exploratory Data Analysis
Data Cleaning: Deduplication
It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data. Following is an example:

In [142]:
a = """ SELECT ProfileName, COUNT(ProfileName) FROM Reviews GROUP BY ProfileName HAVING COUNT(ProfileName) > 1 """

In [143]:
af = pd.read_sql_query(a,connection)

In [144]:
af

,ProfileName,COUNT(ProfileName)
0,,2
1,,2
2,:Jury Duty:,4
3,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",10
4,"""dmab6395""",3
...,...,...
76191,~~~,2
76192,~~~Carol,4
76193,"¢¾ Leah ¢¾ ""Leah""",3
76194,Étaín d'Orphée,6


In [145]:
bf = pd.read_sql_query(""" 
SELECT 
ProfileName, UserId, Score, COUNT(ProfileName) 
FROM Reviews WHERE Score != 3
GROUP BY ProfileName
HAVING COUNT(ProfileName) > 1 """,connection)

In [146]:
bf

,ProfileName,UserId,Score,COUNT(ProfileName)
0,,AVJD277LH9Y4O,1,2
1,,A2JKR0W0EQ6QQM,5,2
2,:Jury Duty:,A2NY8YRUTRWB0Y,5,4
3,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",A1ER6IYOMM8VCT,1,10
4,"""dmab6395""",A34NBH479RB0E,5,3
...,...,...,...,...
71454,~~Trish~~,A364ZQLZSJ3RUH,5,2
71455,~~~,A36OILYUVD4CST,5,2
71456,~~~Carol,A23C6AOWQGUIT3,5,4
71457,Étaín d'Orphée,A16V6UJZPEYTPX,5,6


In [150]:
df3 = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE UserId="A1ER6IYOMM8VCT"
ORDER BY ProductID
""", connection)
df3

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,5386,B000H27K86,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",0,0,5,1199491200,"A TASTE OF ""N'AW'INS"" GREAT SPICE MIX FOR MANY...",I have been using this spice mix for many year...
1,509327,B000PWTQ26,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",2,2,5,1275350400,SOME OF MY BEST FRIENDS ARE DUM DUMS!,"I'm not really a sweet tooth kind of guy, but ..."
2,16831,B001LGGH40,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,2,2,1239580800,TOO SWEET AND TOO SMALL! SWITCH!?? I THINK NOT!,I can't figure out why anyone would want to dr...
3,249620,B001LGGH54,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,2,1,1236124800,TOO SWEET AND SYRUPY FOR MY TASTE!,"Well, if this is all natural you can have it! ..."
4,231983,B002RSVOLM,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,1,5,1269648000,SIMPLY AMAZING!,I am the biggest skeptic when it comes to fads...
5,557551,B003YBLF2E,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",4,6,1,1301961600,OVER PRICED CAT FOOD FOR VERY LAZY PEOPLE!,"OK, I'm all for having some convenience here a..."
6,493790,B0044CPA28,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,1,4,1297900800,"ANOTHER TASTY TREAT, BUT NOTHING NEW HERE EITHER!",I enjoy cereal and this cereal is like most su...
7,55589,B004CYLW7A,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",2,2,4,1297900800,"TASTY, BUT NOTHING NEW!",I enjoy cereal and this cereal is like most su...
8,75788,B004MO6NI8,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,4,1,1331769600,WAY TOO SWEET AND TASTE TERRIBLE!,"I have tried many ""energy drinks"" and try to u..."
9,24455,B004R8J8EK,A1ER6IYOMM8VCT,"! MR. KNOW IT ALL ;-b ""DR SHOCK""",1,2,2,1313625600,"OK, BUT WHAT'S THE POINT!?",Shake N Bake is one of those products that has...


In [151]:
df2 = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", connection)
df2.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [152]:
sorted_data = df.sort_values('ProductId', axis = 0, ascending=True, inplace=False, kind='quicksort',na_position ='last')

In [153]:
sorted_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,Positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,Positive,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,Positive,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,Positive,961718400,Great Product,This was a really good idea and the final prod...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,Positive,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...
...,...,...,...,...,...,...,...,...,...,...
711,765,B009HINRX8,A1OEL4UZT3KKI4,"coffee drinker in PA ""coffee drinker in PA""",0,0,Positive,1344988800,great coffee - terrible price,"This is one of the best choices, in my opinion..."
710,764,B009HINRX8,ADDBLG0CFY9AI,S.A.D.,1,1,Positive,1326758400,Best of the Tassimo's,We've tried many Tassimo flavors. This is by ...
709,763,B009HINRX8,A3N9477PUE6WMR,patc477,4,4,Positive,1323302400,Good Tasting cup o' joe,This is a bold blend that has a great taste. T...
713,768,B009HINRX8,A2CAZG1CQ8BQI5,Patricia J. Nohalty,0,0,Positive,1337212800,Kona for Tassimo,Of all the coffee's available for Tassimo this...


In [159]:
df_final = sorted_data.drop_duplicates(subset = {'UserId','ProfileName','Time','Text'},keep='first',inplace =False)

In [160]:
df_final

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,Positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,Positive,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,Positive,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,Positive,961718400,Great Product,This was a really good idea and the final prod...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,Positive,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...
...,...,...,...,...,...,...,...,...,...,...
711,765,B009HINRX8,A1OEL4UZT3KKI4,"coffee drinker in PA ""coffee drinker in PA""",0,0,Positive,1344988800,great coffee - terrible price,"This is one of the best choices, in my opinion..."
710,764,B009HINRX8,ADDBLG0CFY9AI,S.A.D.,1,1,Positive,1326758400,Best of the Tassimo's,We've tried many Tassimo flavors. This is by ...
709,763,B009HINRX8,A3N9477PUE6WMR,patc477,4,4,Positive,1323302400,Good Tasting cup o' joe,This is a bold blend that has a great taste. T...
713,768,B009HINRX8,A2CAZG1CQ8BQI5,Patricia J. Nohalty,0,0,Positive,1337212800,Kona for Tassimo,Of all the coffee's available for Tassimo this...


# #Checking to see how much % of data still remains

In [161]:
df_final['Id'].size

4986

In [165]:
df['Id'].size

5000

In [171]:
((df_final[ 'Id'].size*1.0)/(df['Id'].size*1.0))*100

99.72

# Observations: Till Now cleaned  the data which contain the Duplicate Values